# 7. Inference on New Images

This notebook runs trained models on **new, unseen images** to demonstrate
real-world applicability of the microbial colony detection pipeline.

**Prerequisites:** Run `1_setup.ipynb` and have trained models available.

## 7.1 Configuration

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor

import config
from utils.visualization import show_single_prediction, setup_metadata_colors

In [ ]:
# ===================== CONFIGURE =====================

# Image to analyze
IMAGE_PATH = "/content/drive/MyDrive/TESE/NEW/20230825_075713951_iOS.jpg"

# Model to use
MODEL_KEY = "faster_rcnn_R50"
TRAINED_MODEL_KEY = "robo_faster_rcnn_R50"  # or any AGAR model key
MODEL_SOURCE = "roboflow"                   # 'agar' or 'roboflow'

# Detection params
NUM_CLASSES = 4          # AGAR: 3, Roboflow: 4
SCORE_THRESHOLD = 0.55
MAX_DETECTIONS = 100

# Dataset name for metadata (class labels/colors)
METADATA_DATASET = "robo_test"  # Must be registered

# ====================================================

print(f"Image: {IMAGE_PATH}")
print(f"Model: {TRAINED_MODEL_KEY} ({MODEL_KEY})")

## 7.2 Build Predictor

In [ ]:
config_file = config.MODELS[MODEL_KEY]
model_weights = config.get_model_weights(TRAINED_MODEL_KEY, MODEL_SOURCE)

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(config_file))
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = False
cfg.MODEL.WEIGHTS = model_weights
cfg.TEST.DETECTIONS_PER_IMAGE = MAX_DETECTIONS

if config.is_retinanet(MODEL_KEY):
    cfg.MODEL.RETINANET.NUM_CLASSES = NUM_CLASSES
    cfg.MODEL.RETINANET.SCORE_THRESH_TEST = SCORE_THRESHOLD
else:
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = NUM_CLASSES
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = SCORE_THRESHOLD

predictor = DefaultPredictor(cfg)
print(f"Predictor ready. Model: {model_weights}")

## 7.3 Run Inference

In [ ]:
show_single_prediction(predictor, IMAGE_PATH, METADATA_DATASET, scale=0.5)

## 7.4 Batch Inference (Optional)

Run inference on all images in a directory.

In [ ]:
# IMAGE_DIR = "/content/drive/MyDrive/TESE/NEW/"
# 
# for fname in sorted(os.listdir(IMAGE_DIR)):
#     if fname.lower().endswith(('.jpg', '.png', '.jpeg')):
#         img_path = os.path.join(IMAGE_DIR, fname)
#         show_single_prediction(predictor, img_path, METADATA_DATASET, scale=0.5)